# sklearn IRIS

Before we start

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# add venv PATH to shell command PATH
import sys, os
if sys.base_prefix not in os.environ['PATH']:
    os.environ['PATH'] = f"{sys.base_prefix}/bin:{os.environ['PATH']}"

In [2]:
import bentoml
NAME = 'sklearn_iris'

# load data & train model

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    
# add parameters for tuning
num_estimators = 100

# train the model
model = RandomForestRegressor(n_estimators=num_estimators)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('predictions: ', predictions)

# log model performance 
mse = mean_squared_error(y_test, predictions)
print("  mse: %f" % mse)

predictions:  [2.   1.   0.   1.   1.67 0.   1.17 1.   0.   1.   2.   1.   0.   2.
 0.   1.88 2.   2.   0.   0.   1.   2.   1.   1.34 1.56 1.88 1.   1.
 2.   2.  ]
  mse: 0.085193


In [4]:
%%writefile {NAME}.py

import bentoml
from bentoml.artifact import SklearnModelArtifact
from bentoml.handlers import DataframeHandler


@bentoml.env()
@bentoml.artifacts([SklearnModelArtifact('model')])
class BentoSvc(bentoml.BentoService):
    @bentoml.api(DataframeHandler)
    def predict(self, inputs):
        outputs = self.artifacts.model.predict(inputs.to_numpy())
        return outputs

Overwriting sklearn_iris.py


In [5]:
from sklearn_iris import BentoSvc

bento_svc = BentoSvc()
bento_svc.pack("model", model)
saved_path = bento_svc.save()
print(saved_path)

[2020-04-01 14:14:04,199] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-04-01 14:14:04,469] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.4.9+175.gc92b567.dirty
creating BentoML-0.4.9+175.gc92b567.dirty/BentoML.egg-info
creating BentoML-0.4.9+175.gc92b567.dirty/benchmark
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/artifact
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/bundler
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/cli
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/clipper
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/configuration
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/configuration/__pycache__
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/deployment
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/deployment/aws_lambda
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/deployment/sagemaker
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/handlers
creating BentoML-0.4.9+175.gc92b567.dirty/bentoml/marshal
creating BentoML-0.4.9+175.gc92b567.dirt

In [12]:
print(saved_path)

/home/ec2-user/bentoml/repository/BentoSvc/20200401141404_4D3C18


# Build & Run Bento Service in Docker

In [6]:
!cd {saved_path}
IMG_NAME = saved_path.split('/')[-1].lower()
!docker build -t {IMG_NAME} {saved_path}

# Optional: set your prefer PYPI mirror
# --build-arg PIP_TRUSTED_HOST=192.168.138.2 \
# --build-arg PIP_INDEX_URL=http://192.168.138.2/simple \

Sending build context to Docker daemon  584.7kB
Step 1/15 : FROM continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/15 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Using cache
 ---> 8835bf4b5d2c
Step 3/15 : EXPOSE 5000
 ---> Using cache
 ---> 4418470d45d9
Step 4/15 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> d26e10d2b55f
Step 5/15 : RUN conda install pip numpy scipy       && pip install gunicorn
 ---> Using cache
 ---> 7c19abe67602
Step 6/15 : COPY . /bento
 ---> 057e7fdd5d43
Step 7/15 : WORKDIR /bento
 ---> Running in 0b31fac6d4f2
Removing intermediate container 0b31fac6d4f2
 ---> 49602d75446a
Step 8/15 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Running in d5bbe552b96f
Removing intermediate container d5bbe552b96f
 ---> cc0a61c23d87
Step 9/15 : RUN conda env update -n base -f /bento/environment.yml
 ---

In [8]:
from bentoml.utils import detect_free_port
PORT = detect_free_port()
print(PORT)
!docker run -itd -p {PORT}:5000 --cpus 1 -e FLAGS="--workers 1 --enable-microbatch" {IMG_NAME}:latest
server_url = f"http://127.0.0.1:{PORT}"
print(server_url)

44723
f0d10d11ab0fd32569108995f5613429036e9fcb88e6aaf459a3fd519702bf3a
http://127.0.0.1:44723


In [20]:
server_url = f"http://127.0.0.1:44723"

# Test with requests

In [21]:
import json
import requests
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

headers = {"content-type": "application/json"}
raw_data = X_test
data = pd.DataFrame(raw_data).to_json()

json_response = requests.post(f'{server_url}/predict',
                              data=data, headers=headers)
print(json_response)
print(json_response.json())

<Response [200]>
[2.0, 1.0, 0.0, 1.0, 1.67, 0.0, 1.17, 1.0, 0.0, 1.0, 2.0, 1.0, 0.0, 2.0, 0.0, 1.88, 2.0, 2.0, 0.0, 0.0, 1.0, 2.0, 1.0, 1.34, 1.56, 1.88, 1.0, 1.0, 2.0, 2.0]


# Benchmark

In [22]:
import pandas as pd
import json
import copy
import random


def get_request_producer():

    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    iris = datasets.load_iris()
    x = iris.data[:, 2:]
    y = iris.target

    url = server_url + '/predict'
    method = "POST"
    headers = {"content-type": "application/json"}
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=7)
    base_data = pd.DataFrame(X_test[:1]).to_dict()

    def _gen_data():
        raw = copy.deepcopy(base_data)
        raw[0][0] += random.random() / 10000
        raw[1][0] += random.random() / 10000
        data = json.dumps(raw)
        return url, method, headers, data

    return _gen_data

# test
get_request_producer()()

('http://127.0.0.1:44723/predict',
 'POST',
 {'content-type': 'application/json'},
 '{"0": {"0": 5.100043360060955}, "1": {"0": 1.800067395442484}}')

In [29]:
from bentoml.utils.benchmark import BenchmarkClient
b = BenchmarkClient(get_request_producer(), lambda: 1, timeout=10)
b.start_session(60, 900, 600)

======= Session started! =======

╒═════════════════╤════════════════╤═══════════════════╤════════════════════╤══════════════════════╕
│   total_request │   failure_rate │   request_per_sec │   avg_request_time │   client_health_rate │
╞═════════════════╪════════════════╪═══════════════════╪════════════════════╪══════════════════════╡
│               0 │              0 │                 0 │                  0 │                    1 │
╘═════════════════╧════════════════╧═══════════════════╧════════════════════╧══════════════════════╛

╒═════════════════╤════════════════╤═══════════════════╤════════════════════╤══════════════════════╕
│   total_request │   failure_rate │   request_per_sec │   avg_request_time │   client_health_rate │
╞═════════════════╪════════════════╪═══════════════════╪════════════════════╪══════════════════════╡
│             939 │              0 │           468.265 │          0.0522521 │                    1 │
╘═════════════════╧════════════════╧═══════════════════╧

In [30]:
b.killall()

# Benchmark with locust

In [ ]:
%%writefile benchmark_{NAME}.py
from locust import HttpLocust, TaskSet, task, constant
from functools import lru_cache

import numpy as np
import pandas as pd
import json


@lru_cache(maxsize=1)
def data_producer():

    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    iris = datasets.load_iris()
    x = iris.data[:, 2:]
    y = iris.target

    headers = {"content-type": "application/json"}
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=7)

    def _gen_data():
        raw = X_test[0][0] + np.random.random([2]) / 10000
        data = pd.DataFrame([raw]).to_json()
        return headers, data

    return _gen_data


class WebsiteTasks(TaskSet):

    @task
    def index(self):
        headers, data = data_producer()()
        self.client.post("/predict", data, headers=headers)

class WebsiteUser(HttpLocust):
    task_set = WebsiteTasks
    wait_time = constant(1)

In [ ]:
!locust -f benchmark_{NAME}.py --slave & locust -f benchmark_{NAME}.py --slave & locust -f benchmark_{NAME}.py --slave & locust -f benchmark_{NAME}.py -H {server_url} --no-web -t 60s --csv {NAME} -c 900 -r 90 --master